### Google Driveをマウント
* Colaboratory上で実行することを想定

In [ ]:
# Google colabを想定
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# google driveにアクセスしたときに見えるフォルダの場所
BASE_PATH = 'drive/MyDrive'

In [ ]:
# その他使用するフォルダの作成
try:
    os.makedirs(f'{BASE_PATH}/oura/data/org')
    os.makedirs(f'{BASE_PATH}/oura/data/org/activity_summary')
    os.makedirs(f'{BASE_PATH}/oura/data/org/bedtime')
    os.makedirs(f'{BASE_PATH}/oura/data/org/daily_activity')
    os.makedirs(f'{BASE_PATH}/oura/data/org/heart_rate')
    os.makedirs(f'{BASE_PATH}/oura/data/org/readiness_summary')
    os.makedirs(f'{BASE_PATH}/oura/data/org/sleep_summary')
    os.makedirs(f'{BASE_PATH}/oura/data/org/tag')
    os.makedirs(f'{BASE_PATH}/oura/data/org/workout')
except FileExistsError:
    pass

In [ ]:
import json
import base64
import urllib.request
from datetime import datetime, timedelta

### トークンの読み込み
* [こちら](https://cloud.ouraring.com/user/sign-in?next=%2Fpersonal-access-tokens)からパーソナルトークンを取得し、「oura_token.json」というファイル名でGoogle Driveにアップしておきます。

In [ ]:
with open(f'{BASE_PATH}/oura_token.json') as f:
    oura_access_token = f.read().strip()

secret = {
    'token': oura_access_token
}

In [ ]:
def request_maker(secret, request, version):
    if version=='v2':
        url = f'https://api.ouraring.com/{version}/usercollection/{request}'
    else:
        url = f'https://api.ouraring.com/{version}/{request}'
    req = urllib.request.Request(url, headers={'Authorization': 'Bearer ' + secret["token"]})
    return req

def request_sender(secret, resource, version):
    req = request_maker(secret, resource, version)
    try:
        with urllib.request.urlopen(req) as response:
            body = json.loads(response.read())
    except urllib.error.URLError as e:
        print(e.reason)        
    return body

### APIから取得する関数（2022/07/26時点）
* 仕様は変更する可能性があります。失敗する場合はAPI Referenceを参照してください。

## Oura API V2
https://cloud.ouraring.com/v2/docs

daily_activity(v2)

In [ ]:
def get_daily_activity(datetime_scope: datetime, path='data/org/daily_activity'):
    datetime_str = datetime_scope.strftime('%Y-%m-%d')
    resource = f'daily_activity?end_date={datetime_str}'
    version = 'v2'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{datetime_str}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

heart rate(v2)

In [ ]:
def get_heart_rate(datetime_scope: datetime, path='data/org/heart_rate'):
    datetime_str = datetime_scope.strftime('%Y-%m-%d')
    resource = f'heartrate?end_date={datetime_str}'
    version = 'v2'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{datetime_str}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

In [ ]:
def get_tag(datetime_scope: datetime, path='data/org/tag'):
    datetime_str = datetime_scope.strftime('%Y-%m-%d')
    resource = f'tag?end_date={datetime_str}'
    version = 'v2'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{datetime_str}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

In [ ]:
def get_workout(datetime_scope: datetime, path='data/org/workout'):
    datetime_str = datetime_scope.strftime('%Y-%m-%d')
    resource = f'workout?end_date={datetime_str}'
    version = 'v2'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{datetime_str}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

## Oura API V1
https://cloud.ouraring.com/docs/

In [ ]:
def get_sleep_summary(datetime_scope: datetime, path='data/org/sleep_summary'):
    start_date = datetime_scope.strftime('%Y-%m-%d')
    end_date = datetime_scope.strftime('%Y-%m-%d')
    resource = f'sleep?start={start_date}&end={end_date}'
    version = 'v1'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{end_date}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

In [ ]:
def get_bedtime(datetime_scope: datetime, path='data/org/bedtime'):
    start_date = datetime_scope.strftime('%Y-%m-%d')
    end_date = datetime_scope.strftime('%Y-%m-%d')
    resource = f'bedtime?start={start_date}&end={end_date}'
    version = 'v1'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{end_date}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

In [ ]:
def get_activity_summary(datetime_scope: datetime, path='data/org/activity_summary'):
    start_date = datetime_scope.strftime('%Y-%m-%d')
    end_date = datetime_scope.strftime('%Y-%m-%d')
    resource = f'activity?start={start_date}&end={end_date}'
    version = 'v1'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{end_date}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

In [ ]:
def get_readiness_summary(datetime_scope: datetime, path='data/org/readiness_summary'):
    start_date = datetime_scope.strftime('%Y-%m-%d')
    end_date = datetime_scope.strftime('%Y-%m-%d')
    resource = f'readiness?start={start_date}&end={end_date}'
    version = 'v1'
    body = request_sender(secret, resource, version)
    with open(f'{path}/{end_date}.json', 'w')as fp:
        json.dump(body, fp, indent=4)
    return body

### APIの取得実行サンプル
* 5分間に5000リクエストが上限です

In [ ]:
import time

nowadays = datetime.now()
for i in range(30):
    yesterday = nowadays - timedelta(i+1)
    get_daily_activity(yesterday)
    get_heart_rate(yesterday)
    get_tag(yesterday)
    get_workout(yesterday)
    get_sleep_summary(yesterday)
    get_bedtime(yesterday)
    get_activity_summary(yesterday)
    get_readiness_summary(yesterday)
    time.sleep(0.1)